In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
  

In [ ]:
from transformers import LlamaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from transformers import LlamaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
!pip uninstall -y tensorflow tensorflow-estimator tensorflow-datasets
!pip install tensorflow-cpu tensorflow-estimator tensorflow-datasets


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [1]:
import pandas as pd
taa_data = pd.read_csv('/content/drive/MyDrive/cti-bench-main/cti-bench-main/data/cti-taa.xls', sep='\t')
vsp_data = pd.read_csv('/content/drive/MyDrive/cti-bench-main/cti-bench-main/data/cti-vsp.xls', sep='\t')
rcm_data = pd.read_csv('/content/drive/MyDrive/cti-bench-main/cti-bench-main/data/cti-rcm.tsv', sep='\t')

In [2]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.2-3B")

OSError: meta-llama/Meta-Llama-3.2-3B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def preprocess_and_split(df, input_column, label_column):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # 80/20 split
    train_dataset = Dataset.from_pandas(train_df[[input_column, label_column]])
    test_dataset = Dataset.from_pandas(test_df[[input_column, label_column]])
    return DatasetDict({'train': train_dataset, 'test': test_dataset})

In [ ]:
taa_dataset = preprocess_and_split(taa_data, "description", "label")  # Adjust columns as per your dataset
vsp_dataset = preprocess_and_split(vsp_data, "description", "score")  # Adjust columns
rcm_dataset = preprocess_and_split(rcm_data, "description", "category")  # Adjust columns

In [ ]:
# Step 1: Mount Google Drive to access your datasets
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install necessary libraries
!pip install transformers datasets

# Step 3: Import necessary modules
from transformers import LlamaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Step 4: Load your datasets from Google Drive
# Adjust the file paths to the location of your datasets in your Google Drive
taa_data = pd.read_csv('/content/drive/MyDrive/datasetcti/cti-bench/data/cti-taa.tsv', sep='\t')
vsp_data = pd.read_csv('/content/drive/MyDrive/datasetcti/cti-bench/data/cti-vsp.tsv', sep='\t')
rcm_data = pd.read_csv('/content/drive/MyDrive/datasetcti/cti-bench/data/cti-rcm.tsv', sep='\t')

# Step 5: Preprocess the datasets for fine-tuning (splitting into 80% training, 20% test)
def preprocess_and_split(df, input_column, label_column):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # 80/20 split
    train_dataset = Dataset.from_pandas(train_df[[input_column, label_column]])
    test_dataset = Dataset.from_pandas(test_df[[input_column, label_column]])
    return DatasetDict({'train': train_dataset, 'test': test_dataset})

taa_dataset = preprocess_and_split(taa_data, "description", "label")  # Adjust columns as per your dataset
vsp_dataset = preprocess_and_split(vsp_data, "description", "score")  # Adjust columns
rcm_dataset = preprocess_and_split(rcm_data, "description", "category")  # Adjust columns

# Step 6: Load the pre-trained LLaMA 8B model for fine-tuning
model = LlamaForSequenceClassification.from_pretrained('huggingface/llama-8b', num_labels=10)

# Step 7: Define training arguments (adjust batch size and epochs according to available resources)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,  # Adjust based on available GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision to save memory
)

# Step 8: Create Trainers for each task
taa_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=taa_dataset['train'],
    eval_dataset=taa_dataset['test'],  # Use the test split here
)

vsp_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=vsp_dataset['train'],
    eval_dataset=vsp_dataset['test'],  # Use the test split here
)

rcm_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=rcm_dataset['train'],
    eval_dataset=rcm_dataset['test'],  # Use the test split here
)

# Step 9: Fine-tune the model for each task
taa_trainer.train()
vsp_trainer.train()
rcm_trainer.train()

# Step 10: Save the model to Google Drive
model.save_pretrained('/content/drive/MyDrive/fine_tuned_llama_8b')
